In [1]:
%load_ext tensorboard

In [2]:

from importlib import reload

from tensorflow.data import AUTOTUNE
import tensorflow_datasets as tfds

import auramask as am

from git import Repo
branch = Repo('./').active_branch.name

from keras_cv.layers import Resizing, Rescaling, Augmenter
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping, Callback

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

from datetime import datetime

2024-02-23 19:53:59.476334: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 19:53:59.513676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 19:53:59.513700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 19:53:59.514629: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 19:53:59.520922: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend


# Inputs

## Victim Models (F)

In [3]:
F = [
  am.models.FaceEmbedEnum.ARCFACE
]

## Image Dataset (X)

In [4]:
ds, info = tfds.load('lfw',
                     decoders=tfds.decode.PartialDecoding({
                       'image': True,
                     }),
                     with_info=True,
                     download=True,
                     as_supervised=False)

2024-02-23 19:54:02.652679: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 19:54:02.687731: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 19:54:02.689182: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
augmenter = Augmenter(
  [
    Rescaling(1./255),
    Resizing(256,256),
  ]
)

def preprocess_data(images, augment=True):
  inputs = {"images": images}
  outputs = augmenter(inputs)
  return outputs['images'], outputs['images']

## Hyperparams

In [6]:
alpha = 2e-4
epsilon = 0.03
l = 0.
BATCH_SIZE = 32
# EPOCH = 500  # ReFace training
EPOCH = 2

# Training

## Training Dataset

In [7]:
train_ds = ds['train']

In [8]:
train_ds = train_ds.batch(BATCH_SIZE).map(
  lambda x: preprocess_data(x['image']),
  num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

### Initialized ATN ($N_{theta}$)

In [9]:
t_loss = am.losses.ReFaceLoss(F=F, l=l)
model = am.models.AuraMask(n_filters=32, n_dims=3, eps=epsilon)

In [10]:
# lpips = am.metrics.PerceptualSimilarity(model=t_loss.lpips.model)
# floss = am.metrics.EmbeddingDistance(F=t_loss.embeddist.F, F_set=t_loss.embeddist.F_set)

model.compile(
  optimizer=Adam(learning_rate=alpha),
  loss=[t_loss.lpips, t_loss.embeddist],
  loss_weights=[l,1.],
  run_eagerly=True
)

In [11]:
SEED = None
for x, _ in train_ds.take(1):
  sample = x

2024-02-23 19:54:08.540215: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [12]:
hparams = {
  hp.HParam('F'): str([f.name.lower() for f in F]),
  hp.HParam('optimizer'): 'adam',
  hp.HParam('alpha'): alpha,
  hp.HParam('epsilon'): epsilon,
  hp.HParam('lambda'): l,
  hp.HParam('Batch Size'): BATCH_SIZE,
}

In [13]:
class ImageCallback(Callback):
  def __init__(self):
    self.epoch = 0
    super().__init__()
  def on_train_begin(self, logs=None):
    hp.hparams(hparams)
    tf.summary.image("Original", sample, max_outputs=10, step=0)
    return super().on_train_begin(logs)
  def on_train_batch_end(self, batch, logs=None):
    if batch % 20 == 0:
      tf.summary.image("Augmented/%d"%self.epoch, self.model(sample), max_outputs=1, step=batch)
    return super().on_train_batch_begin(batch, logs)
  def on_epoch_end(self, epoch, logs=None):
    tf.summary.image("Augmented/epoch", self.model(sample), max_outputs=10, step=epoch)
    self.epoch+=1
    return super().on_epoch_end(epoch, logs)

In [14]:
LOG_DIR = 'logs/nocrop/%s/%s/%s'%(branch, datetime.now().strftime("%Y%m%d"), datetime.now().strftime("%H%M%S"))

In [15]:
tensorboard_callback = TensorBoard(log_dir=LOG_DIR, write_images=True, update_freq=1, histogram_freq=1)
early_stop = EarlyStopping(monitor='loss', patience=3)
img_call = ImageCallback()

In [16]:
training_history = model.fit(
  x=train_ds,
  batch_size=BATCH_SIZE,
  callbacks=[tensorboard_callback, early_stop, img_call],
  epochs=EPOCH,
  verbose=0,
  use_multiprocessing=True
)

2024-02-23 19:54:08.758639: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-23 19:54:09.629918: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-23 19:54:09.698471: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-23 19:54:38.910767: I external/local_xla/xla/service/service.cc:168] XLA service 0x356a9480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-23 19:54:38.910796: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10, Compute Capability 8.6
2024-02-23 19:54:38.915507: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708718079.013081  240343 device_compiler.h:186] Compiled c